# CISC 351/372 Advanced Data Analytics Group Project

## Group 8: Political Sentiment Analysis In Liberal and Conservative Reddit Communities

### RQ2: How does political sentiment differ between liberal and conservative Reddit communities?

This notebook contains the code to answer the second research question.